### Import librairies
Nous importons les bibliothèque fondamentale pour la programme

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Import donnée
Nous allons importer les données des trainSet et testSet dans les features et la variable à déviner

In [13]:
def importData(path, predictColumn=False):
    columnNames=["ECP","RC","MIN","MAX","MEAN","START","END","DURATION","CV","A1","A2","A3","A4","FP"]
    #columnNames=["RC","MIN","MAX","MEAN","START","END","DURATION","CV","A1","A2","A3","A4","FP"]
    delimit=','
    data = pd.read_csv(path, names=columnNames, sep=delimit)
    #data = data.drop(columns="ECP")
    if predictColumn:
        x=data.iloc[:, 1:-1].values
        y=data['FP'].values
        return x, y
    else:
        x=data.iloc[:,:].values
        return x

In [60]:
#pathTrain='../3-DataSet/dataFinale/p25/traintSet_p25.csv'
pathTrain='../3-DataSet/dataFinale/p27/traintSet_p27.csv'
#pathTrain='../3-DataSet/dataFinale/p30/traintSet_p30.csv'
X_train, y_train = importData(pathTrain,True)

#pathTest='../3-DataSet/dataFinale/p25/testSet_p25.csv'
pathTest='../3-DataSet/dataFinale/p27/testSet_p27.csv'
#pathTest='../3-DataSet/dataFinale/p30/testSet_p30.csv'
X_test, y_test = importData(pathTest,True)

### Normalisation et mise en echelle
Avant de faire des prédictions réelles, il est toujours recommandé de mettre à l'échelle les entités afin que toutes puissent être évaluées uniformément. 
L'algorithme de descente de gradient (qui est utilisé dans la formation de réseaux de neurones et d'autres algorithmes d'apprentissage automatique) converge également plus rapidement avec des fonctionnalités normalisées.
![](scaling.JPG)

In [61]:
from sklearn.preprocessing import StandardScaler

def scaleData(data):
    scaler = StandardScaler()
    scaler.fit(data)
    return scaler.transform(data)


X_train = scaleData(X_train)
X_test = scaleData(X_test)

### Classification & prediction
Les machines à vecteurs de support ou séparateurs à vaste marge (en anglais support vector machine, SVM) sont un ensemble de techniques d'apprentissage supervisé destinées à résoudre des problèmes de discrimination et de régression. Les SVM sont une généralisation des classifieurs linéaires.

Les séparateurs à vaste marge ont été développés dans les années 1990 à partir des considérations théoriques de Vladimir Vapnik sur le développement d'une théorie statistique de l'apprentissage : la théorie de Vapnik-Tchervonenkis. Ils ont rapidement été adoptés pour leur capacité à travailler avec des données de grandes dimensions, le faible nombre d'hyperparamètres, leurs garanties théoriques, et leurs bons résultats en pratique.

Les SVM ont été appliqués à de très nombreux domaines (bio-informatique, recherche d'information, vision par ordinateur, finance…). Selon les données, la performance des machines à vecteurs de support est de même ordre, ou même supérieure, à celle d'un réseau de neurones ou d'un modèle de mélanges gaussiens.

In [62]:
from sklearn.svm import SVC

#Classification
svmLinear = SVC(kernel="linear",random_state=0)
svmLinear.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [63]:
#prediction
y_pred_svm_linear = svmLinear.predict(X_test)

## Evaluation
Pour évaluer un algorithme, la matrice de confusion, la précision, le rappel et le score f1 sont les mesures les plus couramment utilisées. Les méthodes confusion_matrixet classification_reportde sklearn.metricspeuvent être utilisées pour calculer ces métriques. Jetez un œil au script suivant:

In [64]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics

def printMetrics(yTest, yPred):
    print("Accuracy: ", metrics.accuracy_score(yTest, yPred))
    print("Ajusted Rand Square",metrics.adjusted_rand_score(yTest, yPred))
    
def showConfusionMatrix(ytest,ypred):
    print("Confusion Matrix \n",confusion_matrix(ytest, ypred))
    print("Repport: \n",classification_report(ytest, ypred))
    
def showScore(classifier, xtest, ytest):
    print('Score: %f' % classifier.score(xtest,ytest))
    print('Erreur: %f' % (1-classifier.score(xtest,ytest)))

In [65]:
showConfusionMatrix(y_test,y_pred_svm_linear)
showScore(svmLinear, X_test,y_test)
printMetrics(y_test, y_pred_svm_linear)

Confusion Matrix 
 [[328   0]
 [ 14  14]]
Repport: 
              precision    recall  f1-score   support

          0       0.96      1.00      0.98       328
          1       1.00      0.50      0.67        28

avg / total       0.96      0.96      0.95       356

Score: 0.960674
Erreur: 0.039326
Accuracy:  0.960674157303
Ajusted Rand Square 0.619402806859


In [66]:
#Classification gaussien
svmRbf = SVC(kernel='rbf',random_state=0)
svmRbf.fit(X_train, y_train)

#prediction
y_pred_rbf = svmRbf.predict(X_test)

showConfusionMatrix(y_test,y_pred_rbf)
showScore(svmRbf, X_test,y_test)
printMetrics(y_test, y_pred_rbf)

Confusion Matrix 
 [[328   0]
 [ 17  11]]
Repport: 
              precision    recall  f1-score   support

          0       0.95      1.00      0.97       328
          1       1.00      0.39      0.56        28

avg / total       0.95      0.95      0.94       356

Score: 0.952247
Erreur: 0.047753
Accuracy:  0.952247191011
Ajusted Rand Square 0.514730434536


In [67]:
#Classification gaussien
svmPoly = SVC(kernel='poly', degree=8,random_state=0)
svmPoly.fit(X_train, y_train)

#prediction
y_predPoly = svmPoly.predict(X_test)

showConfusionMatrix(y_test,y_predPoly)
showScore(svmPoly, X_test,y_test)
printMetrics(y_test, y_predPoly)

Confusion Matrix 
 [[327   1]
 [ 19   9]]
Repport: 
              precision    recall  f1-score   support

          0       0.95      1.00      0.97       328
          1       0.90      0.32      0.47        28

avg / total       0.94      0.94      0.93       356

Score: 0.943820
Erreur: 0.056180
Accuracy:  0.943820224719
Ajusted Rand Square 0.422593652061


In [68]:
#Classification gaussien
svmSig = SVC(kernel='sigmoid',random_state=0)
svmSig.fit(X_train, y_train)

#prediction
y_predSig = svmSig.predict(X_test)

showConfusionMatrix(y_test,y_predSig)
showScore(svmSig, X_test,y_test)
printMetrics(y_test, y_predSig)

Confusion Matrix 
 [[309  19]
 [ 19   9]]
Repport: 
              precision    recall  f1-score   support

          0       0.94      0.94      0.94       328
          1       0.32      0.32      0.32        28

avg / total       0.89      0.89      0.89       356

Score: 0.893258
Erreur: 0.106742
Accuracy:  0.893258426966
Ajusted Rand Square 0.230240266567


### Plot ROC curve
La courbe caractéristique de fonctionnement du récepteur (ROC) est un autre outil commun utilisé avec les classificateurs binaires. La ligne pointillée représente la courbe ROC d'un classificateur purement aléatoire; un bon classificateur reste aussi loin de cette ligne que possible (vers le coin supérieur gauche).

In [11]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

AttributeError: predict_proba is not available when  probability=False

### Predict other Data
Nous allons maintenant tester la prédiction d'autres données qui ne sont pas les tests

In [28]:
#LoadData
path='../3-DataSet/20200122/95/testSets95.csv'

#path='../2-TransformedData/data2_2020-Jan-31-10_55_51.csv'
toPredict = pd.read_csv(path,names=["ECP","RC","MIN","MAX","MEAN","START","END","DURATION","CV","A1","A2","A3","A4"], sep=',')
def scaleData2(data):
    data=data[1:]
    scaler = StandardScaler()
    scaler.fit(data)
    return scaler.transform(data)

#scaled_data = scaleData2(toPredict)

#prediction
new_pred = knn.predict(scaled_data)


#print('Prediction: ',new_pred)


#ShowScore
#showScore(knn, X_test,new_pred)
#print('Score: %f' % knn.score(X_test,new_pred))
#toPredict.columns=["ECP","RC","MIN","MAX","MEAN","START","END","DURATION","CV","A1","A2","A3","A4"]
toPredict['IS_FP']=new_pred
#toPredict.set_option('display.max_rows',73)
toPredict

ValueError: query data dimension must match training data dimension

In [62]:
from sklearn.model_selection import cross_val_score
#create a new KNN model
knn_cv = KNeighborsClassifier(n_neighbors=3)
#train model with cv of 5 
cv_scores = cross_val_score(knn_cv, X_train, y_train, cv=20)
#print each cv score (accuracy) and average them
print('Cross validation score\n',cv_scores)
print('cv_scores mean:{}'.format(np.mean(cv_scores)))


Cross validation score
 [ 1.          0.875       0.75        0.875       0.875       0.75        0.875
  0.75        0.75        0.71428571  0.71428571  0.85714286  0.85714286
  0.85714286  0.85714286  0.85714286  0.85714286  0.85714286  0.85714286
  0.5       ]
cv_scores mean:0.8142857142857144


In [109]:
#Export predict data
pathExport='../6-ClassifiedData/knn_predict_30_2.csv'
toPredict.to_csv(pathExport, index = None, header=True)